In [2]:
import torch
from ultralytics import YOLO
from ultralytics.nn.modules import EConv  # Import your custom layer directly
import traceback

In [3]:
# 1. Test standalone layer
print("Testing standalone EConv:")
try:
    # Create EConv layer directly
    layer = EConv(c1=3, c2=16, k=3, s=1)
    print(f"Layer created: {layer}")

    # Test forward pass
    x = torch.randn(1, 3, 64, 64)
    out = layer(x)
    print(f"✓ Forward pass successful! Input shape: {x.shape}")
    print(f"✓ Output shape: {out.shape}")

    # Verify parameters
    params = sum(p.numel() for p in layer.parameters())
    print(f"✓ Parameters count: {params:,} (expected ~1,000)")
except Exception as e:
    print(f"✗ Test failed: {e}")
    traceback.print_exc()

Testing standalone EConv:
Created EConv: in=3, out=16, k=3, s=1
Layer created: EConv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): SiLU()
  (se): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
    (3): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
    (4): Sigmoid()
  )
)
✓ Forward pass successful! Input shape: torch.Size([1, 3, 64, 64])
✓ Output shape: torch.Size([1, 16, 64, 64])
✓ Parameters count: 513 (expected ~1,000)


In [4]:
print("\nTesting classification model:")
try:
    model = YOLO('yolov8n_econv_classify.yaml', task='classify')
    print("Model loaded successfully!")

    # Verify parameters
    total_params = sum(p.numel() for p in model.model.parameters())
    print(f"Total parameters: {total_params:,}")

    # Test forward pass directly
    dummy = torch.rand(1, 3, 64, 64)  # [0-1] range

    # Use model's forward method
    with torch.no_grad():
        output = model.model(dummy)

    print(f"Output shape: {output.shape}")  # Should be [1, 10]
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()


Testing classification model:
Created EConv: in=3, out=8, k=8, s=8
YOLOv8n_econv_classify summary (fused): 14 layers, 2,259 parameters, 2,259 gradients, 0.0 GFLOPs
Model loaded successfully!
Total parameters: 2,259
Output shape: torch.Size([1, 10])


In [5]:
print(model.model)

ClassificationModel(
  (model): Sequential(
    (0): EConv(
      (conv): Conv2d(3, 8, kernel_size=(8, 8), stride=(8, 8), padding=(4, 4), bias=False)
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
      (se): Sequential(
        (0): AdaptiveAvgPool2d(output_size=1)
        (1): Conv2d(8, 1, kernel_size=(1, 1), stride=(1, 1))
        (2): ReLU()
        (3): Conv2d(1, 8, kernel_size=(1, 1), stride=(1, 1))
        (4): Sigmoid()
      )
    )
    (1): Conv(
      (conv): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): AdaptiveAvgPool2d(output_size=1)
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=8, out_features=10, bias=True)
  )
)


In [7]:
print("\nTesting detection model:")
try:
    # Load model with detection task
    model = YOLO('yolov8n_econv_detect.yaml', task='detect')
    print("Model loaded successfully!")

    # Verify parameters
    total_params = sum(p.numel() for p in model.model.parameters())
    print(f"Total parameters: {total_params:,}")

    # Test forward pass
    dummy = torch.rand(4, 3, 640, 640)  # Standard detection input size

    # Use model's forward method
    with torch.no_grad():
        output = model.model(dummy)

    # Output should be a tuple of feature maps
    print("Output shapes:")
    for i, x in enumerate(output):
        if isinstance(x, torch.Tensor):
            print(f"  Output {i+1}: {x.shape}")
        else:
            print(f"  Output {i+1}: {type(x)}")

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


Testing detection model:
Created EConv: in=3, out=8, k=2, s=16
Created EConv: in=8, out=8, k=2, s=64
Model loaded successfully!
Total parameters: 128,642
Output shapes:
  Output 1: torch.Size([4, 144, 1, 1])
